In [1]:
from braindecode.datasets import BaseConcatDataset, BaseDataset
import os

import numpy as np
import pandas as pd
import mne
from mne.datasets.sleep_physionet.age import fetch_data


from braindecode.datautil.preprocess import preprocess, Preprocessor
from braindecode.datautil.windowers import create_windows_from_events
from braindecode.datautil.preprocess import zscore

import numpy as np
import copy
from tqdm import tqdm
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from scipy.interpolate import interp1d
from sklearn.metrics import (
    cohen_kappa_score,
    accuracy_score,
    f1_score,
    confusion_matrix,
)
from sklearn.linear_model import LogisticRegression as LR

import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import mne
from mne.datasets.sleep_physionet.age import fetch_data

from braindecode.datautil.preprocess import preprocess, Preprocessor
from braindecode.datautil.windowers import create_windows_from_events
from braindecode.datautil.preprocess import zscore
from braindecode.datasets import BaseConcatDataset, BaseDataset

from torch.utils.data import DataLoader
from torch.utils.data.sampler import Sampler
from sklearn.utils import check_random_state

DATA_PATH = '/mnt/scratch'


# Params
NEG_SAMPLES = 128 # number of negative samples per positive sample
BATCH_SIZE = 1
POS_MIN = 1
NEG_MIN = 15
EPOCH_LEN = 7
NUM_SAMPLES = 2000
SUBJECTS = np.arange(10)
RECORDINGS = [1, 2]

random_state = 1234
n_jobs = 1
sfreq = 100
high_cut_hz = 30

window_size_s = 30
sfreq = 100
window_size_samples = window_size_s * sfreq


/home/zombie/miniconda3/envs/torch/lib/python3.8/site-packages/braindecode/datautil/preprocess.py:10: UserWarning: datautil.preprocess module is deprecated and is now under preprocessing.preprocess, please use from import braindecode.preprocessing.preprocess
  warn('datautil.preprocess module is deprecated and is now under '
/home/zombie/miniconda3/envs/torch/lib/python3.8/site-packages/braindecode/datautil/windowers.py:4: UserWarning: datautil.windowers module is deprecated and is now under preprocessing.windowers, please use from import braindecode.preprocessing.windowers
  warn('datautil.windowers module is deprecated and is now under '


In [2]:
random_state = 1234
n_jobs = 1
sfreq = 100
high_cut_hz = 30

window_size_s = 30
sfreq = 100
window_size_samples = window_size_s * sfreq



class SleepPhysionet(BaseConcatDataset):
    def __init__(
        self,
        subject_ids=None,
        recording_ids=None,
        preload=False,
        load_eeg_only=True,
        crop_wake_mins=30,
        crop=None,
    ):
        if subject_ids is None:
            subject_ids = range(83)
        if recording_ids is None:
            recording_ids = [1, 2]

        paths = fetch_data(
            subject_ids,
            recording=recording_ids,
            on_missing="warn",
            path=DATA_PATH,
        )

        all_base_ds = list()
        for p in paths:
            raw, desc = self._load_raw(
                p[0],
                p[1],
                preload=preload,
                load_eeg_only=load_eeg_only,
                crop_wake_mins=crop_wake_mins,
                crop=crop,
            )
            base_ds = BaseDataset(raw, desc)
            all_base_ds.append(base_ds)
        super().__init__(all_base_ds)

    @staticmethod
    def _load_raw(
        raw_fname,
        ann_fname,
        preload,
        load_eeg_only=True,
        crop_wake_mins=False,
        crop=None,
    ):
        ch_mapping = {
            "EOG horizontal": "eog",
            "Resp oro-nasal": "misc",
            "EMG submental": "misc",
            "Temp rectal": "misc",
            "Event marker": "misc",
        }
        exclude = list(ch_mapping.keys()) if load_eeg_only else ()

        raw = mne.io.read_raw_edf(raw_fname, preload=preload, exclude=exclude)
        annots = mne.read_annotations(ann_fname)
        raw.set_annotations(annots, emit_warning=False)

        if crop_wake_mins > 0:
            # Find first and last sleep stages
            mask = [x[-1] in ["1", "2", "3", "4", "R"] for x in annots.description]
            sleep_event_inds = np.where(mask)[0]

            # Crop raw
            tmin = annots[int(sleep_event_inds[0])]["onset"] - crop_wake_mins * 60
            tmax = annots[int(sleep_event_inds[-1])]["onset"] + crop_wake_mins * 60
            raw.crop(tmin=max(tmin, raw.times[0]), tmax=min(tmax, raw.times[-1]))

        # Rename EEG channels
        ch_names = {i: i.replace("EEG ", "") for i in raw.ch_names if "EEG" in i}
        raw.rename_channels(ch_names)

        if not load_eeg_only:
            raw.set_channel_types(ch_mapping)

        if crop is not None:
            raw.crop(*crop)

        basename = os.path.basename(raw_fname)
        subj_nb = int(basename[3:5])
        sess_nb = int(basename[5])
        desc = pd.Series({"subject": subj_nb, "recording": sess_nb}, name="")
       

        return raw, desc


random_state = 1234
n_jobs = -1
sfreq = 100
high_cut_hz = 30

EPOCH_LEN = 15

dataset = SleepPhysionet(
    subject_ids=SUBJECTS, recording_ids=RECORDINGS, crop_wake_mins=30
)

preprocessors = [
    Preprocessor(lambda x: x * 1e6),
    Preprocessor("filter", l_freq=None, h_freq=high_cut_hz, n_jobs=n_jobs),
]

# Transform the data
preprocess(dataset, preprocessors)


mapping = {  # We merge stages 3 and 4 following AASM standards.
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4,
}

windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=0,
    trial_stop_offset_samples=0,
    window_size_samples=window_size_samples,
    window_stride_samples=window_size_samples,
    preload= True,
    mapping=mapping,
)


preprocess(windows_dataset, [Preprocessor(zscore)])


Extracting EDF parameters from /mnt/scratch/physionet-sleep-data/SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/scratch/physionet-sleep-data/SC4002E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/scratch/physionet-sleep-data/SC4011E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/scratch/physionet-sleep-data/SC4012E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/scratch/physionet-sleep-data/SC4021E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/scratch/physionet-sleep-data/SC4022E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info struc

/home/zombie/miniconda3/envs/torch/lib/python3.8/site-packages/braindecode/preprocessing/preprocess.py:52: UserWarning: Preprocessing choices with lambda functions cannot be saved.
  warn('Preprocessing choices with lambda functions cannot be saved.')
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


Reading 0 ... 3348000  =      0.000 ... 33480.000 secs...


[Parallel(n_jobs=6)]: Done   2 out of   2 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=6)]: Done   2 out of   2 | elapsed:    1.5s finished


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 sec)



...

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Adding metadata with 4 columns
Replacing existing metadata with 4 columns
837 matching events found
No baseline correction applied
0 projection items activated
Loading data for 837 events and 3000 original time points ...


[Parallel(n_jobs=6)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=6)]: Done   2 out of   2 | elapsed:    1.0s finished


0 bad epochs dropped
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Adding metadata with 4 columns
Replacing existing metadata with 4 columns
1116 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1116 events and 3000 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Adding metadata with 4 columns
Replacing existing metadata with 4 columns
1088 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1088 events and 3000 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Adding metadata with 4 columns
Replacing existing metadata with 4 columns
1150 matching

/home/zombie/miniconda3/envs/torch/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function zscore is deprecated; will be removed in 0.7.0. Use sklearn.preprocessing.scale instead.
  warnings.warn(msg, category=FutureWarning)


In [3]:
class RelativePositioningDataset(BaseConcatDataset):
    """BaseConcatDataset with __getitem__ that expects 2 indices and a target."""

    def __init__(self, list_of_ds, epoch_len=7):
        super().__init__(list_of_ds)
        self.return_pair = True
        self.epoch_len = epoch_len

    def __getitem__(self, index):

        rec, pos, neg = index
        pos_data = []
        neg_data = []

        # print(rec, pos, neg)

        assert pos != neg, "pos and neg should not be the same"

        for i in range(-(self.epoch_len // 2), self.epoch_len // 2 + 1):
            pos_data.append(super().__getitem__(pos + i)[0])
            neg_data.append(super().__getitem__(neg + i)[0])

        pos_data = np.stack(pos_data, axis=0) # (7, 2, 3000)
        neg_data = np.stack(neg_data, axis=0) # (7, 2, 3000)

        return pos_data, neg_data


class TuneDataset(BaseConcatDataset):
    """BaseConcatDataset for train and test"""

    def __init__(self, list_of_ds):
        super().__init__(list_of_ds)

    def __getitem__(self, index):

        X = super().__getitem__(index)[0]
        y = super().__getitem__(index)[1]

        return X, y


class RecordingSampler(Sampler):
    def __init__(self, metadata, random_state=None, epoch_len=7):

        self.metadata = metadata
        self.epoch_len = epoch_len
        self._init_info()
        self.rng = check_random_state(random_state)

    def _init_info(self):
        keys = ["subject", "recording"]

        self.metadata = self.metadata.reset_index().rename(
            columns={"index": "window_index"}
        )
        self.info = (
            self.metadata.reset_index()
            .groupby(keys)[["index", "i_start_in_trial"]]
            .agg(["unique"])
        )
        self.info.columns = self.info.columns.get_level_values(0)

    def sample_recording(self):
        """Return a random recording index."""
        return self.rng.choice(self.n_recordings)
            
    def __iter__(self):
        raise NotImplementedError

    @property
    def n_recordings(self):
        return self.info.shape[0]


class RelativePositioningSampler(RecordingSampler):
    def __init__(
        self,
        metadata,
        tau_pos,
        tau_neg,
        n_examples,
        same_rec_neg=True,
        random_state=None,
        epoch_len=7,
    ):
        super().__init__(metadata, random_state=random_state, epoch_len=epoch_len)

        self.tau_pos = tau_pos
        self.tau_neg = tau_neg
        self.epoch_len = epoch_len
        self.n_examples = n_examples
        self.same_rec_neg = same_rec_neg
        self.info['index'] = self.info['index'].apply(lambda x: x[self.epoch_len // 2 : -(self.epoch_len // 2) ])
        self.info['i_start_in_trial'] = self.info['i_start_in_trial'].apply(lambda x: x[self.epoch_len // 2 : -(self.epoch_len // 2) ])
        self.info.iloc[-1]['index'] = self.info.iloc[-1]['index'][:-(7 // 2) - 1]
        self.info.iloc[-1]['i_start_in_trial'] = self.info.iloc[-1]['i_start_in_trial'][: -(self.epoch_len // 2) - 1]

    def _sample_pair(self):
        
        """Sample a pair of two windows."""
        # Sample first window
        
        for rec_id in range(self.info.shape[0]):
            epochs = self.info.iloc[rec_id]["index"]
            start_trail = self.info.iloc[rec_id]["i_start_in_trial"]
            for ep_id, trail in zip(epochs, start_trail):

                # print(ep_id, trail)
                win_ind1, rec_ind1 = ep_id, rec_id
                ts1 = trail
                ts = self.info.iloc[rec_ind1]["i_start_in_trial"]

                epoch_min = self.info.iloc[rec_ind1]["i_start_in_trial"][self.epoch_len // 2]
                epoch_max = self.info.iloc[rec_ind1]["i_start_in_trial"][-self.epoch_len // 2]

                if self.same_rec_neg:
                    mask = ((ts <= ts1 - self.tau_neg) & (ts >= epoch_min)) | (
                        (ts >= ts1 + self.tau_neg) & (ts <= epoch_max)
                    )
                    
                if sum(mask) == 0:
                    raise NotImplementedError
                win_ind2 = self.rng.choice(self.info.iloc[rec_ind1]["index"][mask])
                yield rec_ind1, win_ind1, win_ind2

    def __iter__(self):  
        yield from self._sample_pair()
      
    def __len__(self):
        epoch_len = 0
        for rec_id in range(self.info.shape[0]):
            epoch_len += len(self.info.iloc[rec_id]["index"])
        return epoch_len

In [ ]:
a = pretext_sampler
pretext_sampler.info

In [ ]:
# ts1 = self.metadata.iloc[win_ind1]["i_start_in_trial"]
# ts = self.info.iloc[rec_ind1]["i_start_in_trial"]


a.metadata.iloc[525: 535]["i_start_in_trial"]

In [ ]:
a.info.iloc[0]

In [4]:
PRETEXT_PATH = os.path.join(DATA_PATH, "pretext")
TEST_PATH = os.path.join(DATA_PATH, "test")

if not os.path.exists(PRETEXT_PATH): os.mkdir(PRETEXT_PATH)
if not os.path.exists(TEST_PATH): os.mkdir(TEST_PATH)


splitted = dict()

splitted["pretext"] = RelativePositioningDataset(
    [ds for ds in windows_dataset.datasets if ds.description["subject"] in np.arange(3)],
    epoch_len = EPOCH_LEN
)

tau_pos, tau_neg = int(sfreq * POS_MIN * 60), int(sfreq * NEG_MIN * 60)
n_examples_pretext = NUM_SAMPLES * len(splitted["pretext"].datasets)


pretext_sampler = RelativePositioningSampler(
    splitted["pretext"].get_metadata(),
    tau_pos=tau_pos,
    tau_neg=tau_neg,
    n_examples=n_examples_pretext,
    same_rec_neg=True,
    random_state=random_state,  # Same samples for every iteration of dataloader
    epoch_len = 7
)



# Dataloader
pretext_loader = DataLoader(
    splitted["pretext"],
    batch_size=BATCH_SIZE,
    sampler=pretext_sampler
)


for i, arr in tqdm(enumerate(pretext_loader), desc = 'pretext'):
    temp_path = os.path.join(PRETEXT_PATH, str(i) + '.npz')
    np.savez(temp_path, pos = arr[0].numpy().squeeze(0), neg = arr[1].numpy().squeeze(0))

pretext: 6181it [00:37, 163.02it/s]


IndexError: list index out of range

In [ ]:
splitted["pretext"][0, 55, 121]

In [ ]:
from augmentations import *
from loss import loss_fn
from model import sleep_model
from train import *
from utils import *

from braindecode.util import set_random_seeds

import os
import numpy as np
import copy
import wandb
import torch
from torch.utils.data import DataLoader, Dataset


PATH = '/mnt/scratch/sleepkfoldsame/'

# Params
SAVE_PATH = "single-epoch-same.pth"
WEIGHT_DECAY = 1e-4
BATCH_SIZE = 128
lr = 5e-4
n_epochs = 200
NUM_WORKERS = 5
N_DIM = 256
EPOCH_LEN = 7
m = 0.9995

####################################################################################################

random_state = 1234
sfreq = 100
high_cut_hz = 30

# Seeds
rng = np.random.RandomState(random_state)
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False
    print(f"GPU available: {torch.cuda.device_count()}")

set_random_seeds(seed=random_state, cuda=device == "cuda")


##################################################################################################


# Extract number of channels and time steps from dataset
n_channels, input_size_samples = (2, 3000)
model = sleep_model(n_channels, input_size_samples, n_dim = N_DIM)


q_encoder = model.to(device)
k_encoder = model.to(device)

for param_q, param_k in zip(q_encoder.parameters(), k_encoder.parameters()):
    param_k.data.copy_(param_q.data) 
    param_k.requires_grad = False  # not update by gradient

In [ ]:
class TuneDataset(torch.utils.data.Dataset):
    """Dataset for train and test"""

    def __init__(self, subjects):
        self.subjects = subjects
        self._add_subjects()

    def __getitem__(self, index):

        X = self.X[index]
        y =  self.y[index]

        return X, y

    def __len__(self):
        return self.X.shape[0]
        
    def _add_subjects(self):
        self.X = []
        self.y = []
        for subject in self.subjects:
            self.X.append(subject['windows'])
            self.y.append(subject['y'])
        self.X = np.concatenate(self.X, axis=0)
        self.y = np.concatenate(self.y, axis=0)


In [ ]:
keys = ["subject", "recording"]

metadata1 = splitted["train"].get_metadata()
metadata1 = metadata1.reset_index().rename(
    columns={"index": "window_index"}
)
info1 = (
    metadata1.reset_index()
    .groupby(keys)[["index", "i_start_in_trial"]]
    .agg(["unique"])
)
info1.columns = info1.columns.get_level_values(0)

rng = np.random.RandomState(1234)
rec_ind1 = rng.choice(info1.shape[0])

In [ ]:
keys = ["subject", "recording"]

metadata = splitted["pretext"].get_metadata()
metadata = metadata.reset_index().rename(
    columns={"index": "window_index"}
)
info = (
    metadata.reset_index()
    .groupby(keys)[["index", "i_start_in_trial"]]
    .agg(["unique"])
)

info.columns = info.columns.get_level_values(0)
epoch_len = 7
rng = np.random.RandomState(1234)
rec_ind = rng.choice(info.shape[0])
win_ind = rng.choice(
        info.iloc[rec_ind]["index"][epoch_len // 2 : -epoch_len // 2]
    )

In [ ]:
class TuneDataset(BaseConcatDataset):
    """BaseConcatDataset for train and test"""

    def __init__(self, list_of_ds):
        super().__init__(list_of_ds)

    def __getitem__(self, index):

        X = super().__getitem__(index)[0]
        y = super().__getitem__(index)[1]

        return X, y

splitted["train"] = TuneDataset(
    [ds for ds in windows_dataset.datasets if ds.description["subject"] in sub_train]
)

splitted["test"] = TuneDataset(
    [ds for ds in windows_dataset.datasets if ds.description["subject"] in sub_test]
)

In [ ]:
splitted["train"].get_metadata()

In [ ]:
splitted["pretext"].get_metadata()

In [ ]:
%matplotlib inline

from multi_epoch.utils import CosineAnnealingWarmupRestarts
import matplotlib.pyplot as plt
import torch
import torchvision.models as models

model = models.resnet18()

a = torch.randn((10, 10), requires_grad=True)
b = torch.randn((10, 10), requires_grad=True)
c = a+b

optimizer = torch.optim.Adam(model.parameters())
scheduler = CosineAnnealingWarmupRestarts(optimizer, first_cycle_steps = 500, cycle_mult = 1.2, max_lr = 0.01, min_lr = 5e-6, warmup_steps = 100, gamma = 0.5)

step = 0
lrs = []

for i in range(10000):
    step += 1
    optimizer.step()
    lrs.append(optimizer.param_groups[0]["lr"])
    scheduler.step()
    
plt.plot(lrs)


In [ ]:
import argparse
import wandb
import numpy as np
import pytorch_lightning as pl
import os
from sklearn.model_selection import KFold
from pytorch_lightning.callbacks import LearningRateMonitor
from data_preprocessing.dataloader import data_generator,cross_data_generator,ft_data_generator
from trainer import sleep_ft,sleep_pretrain
from config import Config
#path = "/scratch/SLEEP_data/data_multi/sleepEDF/"
path = "/scratch/SLEEP_data/"
training_mode = 'ss'
config = Config()
def ft_fun(file_name,epoch):
    file_name = file_name+"_epoch"+str(epoch)+'.pt'
    name = os.path.join(config.exp_path,file_name)
    src_path = '/scratch/SLEEP_data/'
    n = cross_data_generator(src_path,[],[],config)
    kfold = KFold(n_splits=5,shuffle=False)
    idxs = np.arange(0,n,1)
    for split,(train_idx,val_idx) in enumerate(kfold.split(idxs)):
        wandb.init(project='delete',group='K-Cross LE',job_type='split: '+str(split),notes='',name=file_name,save_code=True,entity='sleep-staging')
        train_dl,valid_dl= cross_data_generator(src_path,train_idx,val_idx,config)
        le_model = sleep_ft(name,config,train_dl,valid_dl,wandb)
        lr_monitor = LearningRateMonitor(logging_interval='step')
        le_trainer = pl.Trainer(callbacks=[lr_monitor],profiler='simple',enable_checkpointing=False,max_epochs=config.num_ft_epoch,gpus=1)
        wandb.watch([le_model],log='all',log_freq=200)
        le_trainer.fit(le_model)
        wandb.finish(quiet=True)
parser = argparse.ArgumentParser()
parser.add_argument("--file_name", type=str, default="finv1_fusion_intra",
                    help="weights file name")
parser.add_argument("--epoch", type= int, default=20,
                    help="current epoch")
args = parser.parse_args()
ft_fun(args.file_name,int(args.epoch))

In [ ]:
def cross_data_generator(data_path,train_idxs,val_idxs,configs):
    train_ds = torch.load(os.path.join(data_path, "train.pt"))
    train_ds['samples'] = train_ds['samples'][:,0,:].unsqueeze(1)
    valid_ds = torch.load(os.path.join(data_path, "val.pt"))
    valid_ds['samples'] = valid_ds['samples'][:,0,:].unsqueeze(1)
    train_subs = [48,72,24,30,34,50,38,15,60,12]
    train_segs = [3937, 2161, 3448, 1783, 3083, 2429, 3647, 2714, 3392, 2029]
    val_subs = [23,26,37,44,49,51,54,59,73,82]
    val_segs = [2633, 2577, 2427, 2287, 2141, 2041, 2864, 3071, 4985, 3070]
    segs = train_segs+val_segs
    if train_idxs !=[]:
        dataset = {}
        train_dataset = {}
        valid_dataset = {}
        dataset['samples'] = torch.from_numpy(np.vstack((train_ds['samples'],valid_ds['samples'])))
        dataset['labels'] = torch.from_numpy(np.hstack((train_ds['labels'],valid_ds['labels'])))
        dataset['samples'] = torch.split(dataset['samples'],segs)
        dataset['labels'] = torch.split(dataset['labels'],segs)
        print('Split Shape',len(dataset['samples']))
        train_dataset['samples'] = [dataset['samples'][i] for i in train_idxs]
        train_dataset['labels'] = [dataset['labels'][i] for i in train_idxs]
        dataset['samples'] = torch.cat(dataset['samples'])
        dataset['labels'] = torch.cat(dataset['labels'])
        print('Stack Shape',dataset['samples'].shape,dataset['labels'].shape)
        print('Train Shape',train_dataset['samples'].shape,train_dataset['labels'].shape)
        train_dataset['samples'] = torch.cat(train_dataset['samples'])
        train_dataset['labels'] = torch.cat(train_dataset['labels'])
        print('Train Shape',train_dataset['samples'].shape,train_dataset['labels'].shape)
        train_dataset = Load_Dataset(train_dataset, configs,data_path=data_path,training_mode='ft')
        valid_dataset['samples'] = [dataset['samples'][i] for i in val_idxs]
        valid_dataset['labels'] = [dataset['labels'][i] for i in val_idxs]
        valid_dataset['samples'] = torch.cat(valid_dataset['samples'])
        valid_dataset['labels'] = torch.cat(valid_dataset['labels'])
        valid_dataset = Load_Dataset(valid_dataset,configs,data_path=data_path,training_mode='ft')
        train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=configs.batch_size,
                                                   shuffle=True, drop_last=configs.drop_last,
                                                   num_workers=10,pin_memory=True,persistent_workers=True)
        valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=configs.batch_size,
                                                   shuffle=False, drop_last=configs.drop_last,
                                                   num_workers=10,pin_memory=True,persistent_workers=True)
        del dataset
        del train_dataset
        del valid_dataset
        return train_loader,valid_loader
    ret = len(val_subs)+len(train_subs)
    del train_ds
    del valid_ds
    return ret

In [ ]:
from braindecode.util import set_random_seeds
from multi_epoch.utils import *
import os
import numpy as np
import copy
import wandb
import torch
from torch.utils.data import DataLoader, Dataset


PATH = '/scratch/sleepkfoldsame/'

# Params
SAVE_PATH = "multi-epoch-kfold.pth"
WEIGHT_DECAY = 1e-4
BATCH_SIZE = 128
lr = 5e-4
n_epochs = 250
NUM_WORKERS = 5
N_DIM = 128
EPOCH_LEN = 7


class pretext_data(Dataset):

    def __init__(self, filepath):
        
        self.file_path = filepath
        self.idx = np.array(range(len(self.file_path)))

    def __len__(self):
        return len(self.file_path)

    def __getitem__(self, index):
        
        path = self.file_path[index]
        data = np.load(path)
        pos = data['pos']
        neg = data['neg']
        anc = copy.deepcopy(pos)
        
        for i in range(pos.shape[0]):
            pos[i] = augment(pos[i])
            anc[i] = augment(anc[i])
            neg[i] = augment(neg[i])
       
        return anc, pos, neg
    
class train_data(Dataset):

    def __init__(self, filepath):
        
        self.file_path = filepath
        self.idx = np.array(range(len(self.file_path)))

    def __len__(self):
        return len(self.file_path)

    def __getitem__(self, index):
        
        path = self.file_path[index]
        data = np.load(path)
        
        return data['x'], data['y']
    
    

PRETEXT_FILE = os.listdir(os.path.join(PATH, "pretext"))
PRETEXT_FILE.sort(key=natural_keys)
PRETEXT_FILE = [os.path.join(PATH, "pretext", f) for f in PRETEXT_FILE]

TEST_FILE = os.listdir(os.path.join(PATH, "test"))
TEST_FILE.sort(key=natural_keys)
TEST_FILE = [os.path.join(PATH, "test", f) for f in TEST_FILE]

print(f'Number of pretext files: {len(PRETEXT_FILE)}')
print(f'Number of test records: {len(TEST_FILE)}')

pretext_loader = DataLoader(pretext_data(PRETEXT_FILE), batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

test_records = [np.load(f) for f in TEST_FILE]
test_subjects = dict()

for i, rec in enumerate(test_records):
    if rec['_description'][0] not in test_subjects.keys():
        test_subjects[rec['_description'][0]] = [rec]
    else:
        test_subjects[rec['_description'][0]].append(rec)

test_subjects = list(test_subjects.values())

In [ ]:
import sys

sys.path.append('/home/zombie/multi-epoch/single_epoch_momentum')

from augmentations import *
from loss import loss_fn
from model import sleep_model
from train import *
from utils import *

from braindecode.util import set_random_seeds

import os
import numpy as np
import copy
import wandb
import torch
from torch.utils.data import DataLoader, Dataset


PATH = '/mnt/scratch/sleepkfoldsame/'

# Params
SAVE_PATH = "single-epoch-same.pth"
WEIGHT_DECAY = 1e-4
BATCH_SIZE = 128
lr = 5e-4
n_epochs = 200
NUM_WORKERS = 5
N_DIM = 256
EPOCH_LEN = 7
m = 0.9995

####################################################################################################

random_state = 1234
sfreq = 100
high_cut_hz = 30

# Seeds
rng = np.random.RandomState(random_state)
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False
    print(f"GPU available: {torch.cuda.device_count()}")

set_random_seeds(seed=random_state, cuda=device == "cuda")


##################################################################################################


# Extract number of channels and time steps from dataset
n_channels, input_size_samples = (2, 3000)
model = sleep_model(n_channels, input_size_samples, n_dim = N_DIM)


q_encoder = model.to(device)
k_encoder = model.to(device)

# for param_q, param_k in zip(q_encoder.parameters(), k_encoder.parameters()):
#     param_k.data.copy_(param_q.data) 
#     param_k.requires_grad = False  # not update by gradient


In [ ]:
#scaling augmentation test
import os
import numpy as np
PATH =  "/mnt/scratch/sleepkfoldsame/test/731.npz"
data = np.load(PATH)

In [ ]:
data['windows'][0,0].shape

In [ ]:
def scaling(x,degree_scale=2):
    #eprint(x.shape)
    ret = np.zeros_like(x)
    degree = 0.05*(degree_scale+np.random.rand())
    factor = 2*np.random.normal(size=x.shape[1])-1
    factor = 1.5+(2*np.random.rand())+degree*factor
    for i in range(x.shape[0]):
        ret[i]=x[i]*factor
    ret = torch.from_numpy(ret)
    return ret 

In [ ]:
import matplotlib.pyplot as plt
import torch

original = data['windows'][0]
scaled = scaling(original)
plt.figure(figsize = (25, 10))
plt.plot(original[0])
plt.show()

In [ ]:
plt.figure(figsize = (25, 10))
plt.plot(scaled[0])
plt.show()

In [ ]:
def noise_channel(ts, mode, degree):
    """
    Add noise to ts
    
    mode: high, low, both
    degree: degree of noise, compared with range of ts    
    
    Input:
        ts: (n_length)
    Output:
        out_ts: (n_length)
        
    """
    len_ts = len(ts)
    num_range = np.ptp(ts)+1e-4 # add a small number for flat signal
    
    ### high frequency noise
    if mode == 'high':
        noise = degree * num_range * (2*np.random.rand(len_ts)-1)
        out_ts = ts + noise
        
    ### low frequency noise
    elif mode == 'low':
        noise = degree * num_range * (2*np.random.rand(len_ts//100)-1)
        x_old = np.linspace(0, 1, num=len_ts//100, endpoint=True)
        x_new = np.linspace(0, 1, num=len_ts, endpoint=True)
        f = interp1d(x_old, noise, kind='linear')
        noise = f(x_new)
        out_ts = ts + noise
        
    ### both high frequency noise and low frequency noise
    elif mode == 'both':
        noise1 = degree * num_range * (2*np.random.rand(len_ts)-1)
        noise2 = degree * num_range * (2*np.random.rand(len_ts//100)-1)
        x_old = np.linspace(0, 1, num=len_ts//100, endpoint=True)
        x_new = np.linspace(0, 1, num=len_ts, endpoint=True)
        f = interp1d(x_old, noise2, kind='linear')
        noise2 = f(x_new)
        out_ts = ts + noise1 + noise2

    else:
        out_ts = ts

    return out_ts

def jitter(x, degree_scale=1):

    #mode = np.random.choice(['high', 'low', 'both', 'no'])
    mode = 'both' 

    ret = []
    for chan in range(x.shape[0]):
        ret.append(noise_channel(x[chan],mode,0.05*degree_scale))
    ret = np.vstack(ret)
    ret = torch.from_numpy(ret)
    return ret 


In [ ]:
import numpy as np
import torch
from scipy.interpolate import interp1d

jittered = jitter(original)


plt.figure(figsize = (25, 10))
plt.plot(original[0])
plt.show()

plt.figure(figsize = (25, 10))
plt.plot(jittered[0])
plt.show()